In [12]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain.document_loaders import PyPDFLoader
import os
from langchain_openai import OpenAIEmbeddings
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
load_dotenv()

True

In [3]:
llm = AzureChatOpenAI(openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"))

In [4]:
file_path = "policy_data.pdf"
loader = PyPDFLoader(file_path)
pages = loader.load()

In [5]:
splitter = CharacterTextSplitter(
    chunk_size=700,
    length_function=len,
)

In [6]:
for document in pages:
    document.metadata['file_name'] = document.metadata['source']

In [13]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [16]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.from_langchain(
    generator_llm = llm,
    critic_llm = llm,
    embeddings= embeddings,
)

In [18]:
testset = generator.generate_with_langchain_docs(pages, test_size=10, distributions={simple: 0.25, reasoning: 0.50, multi_context: 0.25})

embedding nodes:   0%|          | 0/38 [00:00<?, ?it/s]

Exception in thread Thread-32:
Traceback (most recent call last):
  File "/Users/siddartha/miniforge3/lib/python3.10/site-packages/openai/_base_client.py", line 1522, in _request
    response = await self._client.send(
  File "/Users/siddartha/miniforge3/lib/python3.10/site-packages/httpx/_client.py", line 1661, in send
    response = await self._send_handling_auth(
  File "/Users/siddartha/miniforge3/lib/python3.10/site-packages/httpx/_client.py", line 1689, in _send_handling_auth
    response = await self._send_handling_redirects(
  File "/Users/siddartha/miniforge3/lib/python3.10/site-packages/httpx/_client.py", line 1726, in _send_handling_redirects
    response = await self._send_single_request(request)
  File "/Users/siddartha/miniforge3/lib/python3.10/site-packages/httpx/_client.py", line 1763, in _send_single_request
    response = await transport.handle_async_request(request)
  File "/Users/siddartha/miniforge3/lib/python3.10/site-packages/httpx/_transports/default.py", line 3

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

Task was destroyed but it is pending!
task: <Task pending name='Task-326' coro=<as_completed.<locals>.sema_coro() done, defined at /Users/siddartha/miniforge3/lib/python3.10/site-packages/ragas/executor.py:35> wait_for=<_GatheringFuture pending cb=[Task.task_wakeup()]> cb=[as_completed.<locals>._on_completion() at /Users/siddartha/miniforge3/lib/python3.10/asyncio/tasks.py:558]>
Task was destroyed but it is pending!
task: <Task pending name='Task-327' coro=<as_completed.<locals>.sema_coro() done, defined at /Users/siddartha/miniforge3/lib/python3.10/site-packages/ragas/executor.py:35> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[as_completed.<locals>._on_completion() at /Users/siddartha/miniforge3/lib/python3.10/asyncio/tasks.py:558]>
Task was destroyed but it is pending!
task: <Task pending name='Task-329' coro=<as_completed.<locals>.sema_coro() done, defined at /Users/siddartha/miniforge3/lib/python3.10/site-packages/ragas/executor.py:35> wait_for=<_GatheringFuture pending c

In [20]:
data_ragas = testset.to_pandas()

In [21]:
data_ragas.to_csv("data_testing_ragas.csv", index=False)